In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
cd '/content/drive/MyDrive/Colab Notebooks/2022-1 DA 시니어 프로젝트 3차'

/content/drive/MyDrive/Colab Notebooks/2022-1 DA 시니어 프로젝트 3차


In [5]:
df = pd.read_csv("./웰니스_대화_스크립트_데이터셋.csv")
df.head(5)

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


## Define functions

In [6]:
!pip install pororo

     |████████████████████████████████| 256 kB 5.4 MB/s 
     |████████████████████████████████| 748.8 MB 18 kB/s 
     |████████████████████████████████| 42.4 MB 1.5 MB/s 
     |████████████████████████████████| 4.0 MB 28.8 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 1.2 MB 31.6 MB/s 
     |████████████████████████████████| 1.7 MB 34.4 MB/s 
     |████████████████████████████████| 1.5 MB 40.3 MB/s 
     |████████████████████████████████| 3.1 MB 34.9 MB/s 
     |████████████████████████████████| 468 kB 44.4 MB/s 
     |████████████████████████████████| 5.9 MB 23.7 MB/s 
     |████████████████████████████████| 90 kB 8.7 MB/s 
     |████████████████████████████████| 147 kB 46.6 MB/s 
     |████████████████████████████████| 749 kB 46.4 MB/s 
     |████████████████████████████████| 1.2 MB 44.7 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 34.9 MB/s 
     |███████████████

In [7]:
from pororo import Pororo
mt = Pororo(task="translation", lang="multi")

In [8]:
def paraphrase(text) :
    text_to_eng = mt(text, src="ko", tgt="en")
    text_to_kor = mt(text_to_eng, src="en", tgt="ko")
    return text_to_kor

In [9]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 47.2 MB/s 


In [10]:
import pandas as pd
import numpy as np
import random
import konlpy
import re
from konlpy.tag import Komoran
komoran = Komoran()

In [11]:
df1 = df.copy()
df1.drop('유저', axis=1, inplace= True)
df1.drop('Unnamed: 3', axis=1, inplace= True)
df1 = df1.dropna(axis=0)
df1

,구분,챗봇
0,감정/감정조절이상,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.
...,...,...
5196,치료이력/응급실,응급실이 있어서 다행이네요. 큰 문제는 없으신 거죠?
5197,치료이력/응급실,응급실에 가셨다니 정말 놀랐어요. 아무 문제 없으신가요? 걱정 되네요.
5213,현재상태/증상악화,상태가 더 안 좋아지셨군요. 걱정이 되네요.
5214,현재상태/증상악화,정말 힘드시겠어요. 지금도 증상이 심하신가요?


In [12]:
change_pronoun = pd.DataFrame()
change_pronoun['Na_list'] = ['나는', '나도', '내', '나의', '내가']
change_pronoun['DangShin_list'] = ['당신은', '당신도', '당신의', '당신의', '당신이']

In [13]:
change_pronoun

,Na_list,DangShin_list
0,나는,당신은
1,나도,당신도
2,내,당신의
3,나의,당신의
4,내가,당신이


In [14]:
def change_eomi_and_pronoun (sentence):
    string = re.findall(r'\w+', sentence)

    if string != [] :
        #pronoun change
        subject_to_pronoun_change = string[:-1]
        pronoun_changed_sentence = []
        for tokens in subject_to_pronoun_change:
            if tokens in ['나는', '나도', '내', '나의', '내가']:
                get_index = change_pronoun['Na_list'][change_pronoun['Na_list'] == tokens].index
                index = int(*get_index)
                tokens = change_pronoun['DangShin_list'][index]
            pronoun_changed_sentence.append(tokens)

        #eomi change
        word = string[-1]
        morphtags = Komoran().pos(word)

        if morphtags[0][1] == 'VA' or morphtags[0][1] == 'VV' or morphtags[0][1] == 'VXV' or morphtags[0][1] == 'VXA' or morphtags[0][1] == 'VCP' or morphtags[0][1] == 'VCN': #lemmatize
            changed_word = str(morphtags[0][0] + '군요.')
            changed_sentence = ' '.join(pronoun_changed_sentence) + ' ' + changed_word
            return changed_sentence

        else :
            candidate = ['그런 일이 있으셨군요.', '그랬군요.', '그러셨군요.']
            sentence = np.random.choice(candidate)

    return sentence

In [15]:
def response(category, text):

    # reflection 
        # 평서문
    if '?' not in text:
         paraphrased_text = paraphrase(text)

         if '.' in paraphrased_text: # 가장 마지막 문장에 대해서만 reflection
             string = paraphrased_text.split('.') 
             paraphrased_text = string[-2]

         reflection_text = change_eomi_and_pronoun(paraphrased_text)

        # 의문문
    else:
        reflection_text = ''

    # additional comment
    get_index = df1['챗봇'][df1['구분'] == category].index
    candidate = [*get_index]
    if candidate != []:
        additional_comment = (df1['챗봇'][np.random.choice(candidate)])
    else :
        additional_comment = ''

    return reflection_text + ' ' + additional_comment

## Response Generation

In [16]:
df2 = pd.read_csv('./df2.csv')
df2

,구분,유저,챗봇,response
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,남편을 보면 화가 나서 감정을 조절할 수 없군요. 감정이 조절이 안 될 때만큼 힘들...
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,당신은 더 이상 감정을 통제할 수 없군요. 감정이 조절이 안 될 때만큼 힘들 때는 ...
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",당신은 하루 종일 롤러코스터를 타는 것 같군요. 저도 그 기분 이해해요. 많이 힘드시죠?
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,당신은 롤러코스터를 타고 있는 것 같군요. 감정이 조절이 안 될 때만큼 힘들 때는 ...
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,당신은 롤러코스터를 타는 것 같군요. 저도 그 기분 이해해요. 많이 힘드시죠?
...,...,...,...,...
5226,현재상태/증상지속,한숨 자고 일어나면 괜찮으려나? 해서 자고 일어났는데도 똑같아요.,NaN,0
5227,현재상태/증상지속,자고 나면 괜찮을 줄 알았는데 비슷해…,NaN,0
5228,현재상태/증상지속,지금도 상태가 계속 안 좋아.,NaN,0
5229,현재상태/증상지속,이게 제 마음과 상관없이 증상이 계속 나타나요.,NaN,0


In [17]:
from tqdm.notebook import tqdm

In [21]:
for i in tqdm(range(4490, 4500)): 
    df2['response'][i] = response(df2['구분'][i], df2['유저'][i]) 
    if i % 10 == 0:
        df2.to_csv('./df2.csv',index=False, encoding='utf-8')
        print(f'iteration: {i} | saved')

  0%|          | 0/10 [00:00<?, ?it/s]

iteration: 4490 | saved


In [25]:
df3 = pd.read_csv('./df2.csv')
df3

,구분,유저,챗봇,response
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,남편을 보면 화가 나서 감정을 조절할 수 없군요. 감정이 조절이 안 될 때만큼 힘들...
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,당신은 더 이상 감정을 통제할 수 없군요. 감정이 조절이 안 될 때만큼 힘들 때는 ...
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",당신은 하루 종일 롤러코스터를 타는 것 같군요. 저도 그 기분 이해해요. 많이 힘드시죠?
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,당신은 롤러코스터를 타고 있는 것 같군요. 감정이 조절이 안 될 때만큼 힘들 때는 ...
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,당신은 롤러코스터를 타는 것 같군요. 저도 그 기분 이해해요. 많이 힘드시죠?
...,...,...,...,...
5226,현재상태/증상지속,한숨 자고 일어나면 괜찮으려나? 해서 자고 일어났는데도 똑같아요.,NaN,
5227,현재상태/증상지속,자고 나면 괜찮을 줄 알았는데 비슷해…,NaN,그러셨군요.
5228,현재상태/증상지속,지금도 상태가 계속 안 좋아.,NaN,당신은 여전히 컨디션이 안 좋군요.
5229,현재상태/증상지속,이게 제 마음과 상관없이 증상이 계속 나타나요.,NaN,이는 당신의 마음에 상관없이 증상이 계속 나타나고 있군요.


In [26]:
for i in range(len(df3)):
    if df3['response'][i] =='0':
        print(i)

In [24]:
df2.to_csv('./df2.csv',index=False, encoding='utf-8')